## SMO

In [1]:
def loadDataSet(fileName):
    dataMat = []
    labelMat = []
    fr = open(fileName).readlines()
    for line in fr:
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat, labelMat

In [2]:
import numpy as np

In [3]:
def selectJrand(i,m):
    j = i
    while j == i:
        j = int(np.random.uniform(0, m))
    return j

In [4]:
def clipAlpha(aj, H, L):
    if aj > H: 
        aj = H
    if L > aj:
        aj = L
    return aj

In [5]:
dataArr, labelArr = loadDataSet('testSet.txt')

In [6]:
labelArr

[-1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0]

### 简化版SMO

In [7]:
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    dataMatrix = np.mat(dataMatIn)
    labelMat = np.mat(classLabels).T
    b = 0 
    m, n = np.shape(dataMatrix)
    alphas = np.mat(np.zeros((m, 1)))
    iter = 0
    while iter < maxIter:
        alphaPairsChanged = 0
        for i in range(m):
            fXi = float(np.multiply(alphas, labelMat).T * (dataMatrix * dataMatrix[i, :].T)) + b
            Ei = fXi - float(labelMat[i])
            if ((labelMat[i] * Ei < -toler) and (alphas[i] < C)) or ((labelMat[i] * Ei > toler) and (alphas[i] > 0)):
                j = selectJrand(i,m)
                fXj = float(np.multiply(alphas,labelMat).T * (dataMatrix*dataMatrix[j, :].T)) + b
                Ej = fXj - float(labelMat[j])
                alphaIold = alphas[i].copy() 
                alphaJold = alphas[j].copy()
                if not labelMat[i] == labelMat[j]:
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L==H: 
                    print("L==H")
                    continue
                eta = 2.0 * dataMatrix[i, :] * dataMatrix[j, :].T - dataMatrix[i, :] * dataMatrix[i, :].T - dataMatrix[j, :] * dataMatrix[j, :].T
                if eta >= 0: 
                    print("eta>=0")
                    continue
                alphas[j] -= labelMat[j] * (Ei - Ej) / eta
                alphas[j] = clipAlpha(alphas[j], H, L)
                if abs(alphas[j] - alphaJold) < 0.00001: 
                    print("j not moving enough")
                    continue
                alphas[i] += labelMat[j] * labelMat[i] * (alphaJold - alphas[j])
                b1 = b - Ei - labelMat[i] * (alphas[i] - alphaIold) * dataMatrix[i, :] * dataMatrix[i, :].T - labelMat[j] * (alphas[j] - alphaJold) * dataMatrix[i, :] * dataMatrix[j, :].T
                b2 = b - Ej - labelMat[i] * (alphas[i] - alphaIold) * dataMatrix[i, :] * dataMatrix[j, :].T - labelMat[j] * (alphas[j] - alphaJold) * dataMatrix[j, :]*dataMatrix[j, :].T
                if (0 < alphas[i]) and (C > alphas[i]): 
                    b = b1
                elif (0 < alphas[j]) and (C > alphas[j]): 
                    b = b2
                else: 
                    b = (b1 + b2) / 2.0
                alphaPairsChanged += 1
                print("iter: %d i:%d, pairs changed %d" % (iter, i, alphaPairsChanged))
        if (alphaPairsChanged == 0): 
            iter += 1
        else: 
            iter = 0
        print("iteration number: %d" % iter)
    return b, alphas

In [8]:
b, alphas = smoSimple(dataArr, labelArr, 0.6, 0.001, 40)

L==H
L==H
iter: 0 i:2, pairs changed 1
j not moving enough
j not moving enough
iter: 0 i:7, pairs changed 2
iter: 0 i:8, pairs changed 3
L==H
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
L==H
L==H
j not moving enough
L==H
L==H
L==H
iter: 0 i:52, pairs changed 4
L==H
L==H
j not moving enough
j not moving enough
j not moving enough
iter: 0 i:69, pairs changed 5
j not moving enough
j not moving enough
j not moving enough
L==H
L==H
j not moving enough
iteration number: 0
L==H
j not moving enough
j not moving enough
j not moving enough
iter: 0 i:6, pairs changed 1
iter: 0 i:7, pairs changed 2
iter: 0 i:8, pairs changed 3
j not moving enough
L==H
iter: 0 i:25, pairs changed 4
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iter: 0 i:55, pairs changed 5
j not moving enough
L==H
iter: 0 i:69, pairs changed 6
iter: 0 i:79, pairs changed 7
j not moving enough
iter: 0 i:85, pairs changed 8
j not moving eno

j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 4
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 5
iter: 5 i:10, pairs changed 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
L==H


j not moving enough
iter: 1 i:69, pairs changed 3
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
iter: 0 i:17, pairs changed 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
iter: 2 i:10, pairs changed 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not movi

iteration number: 15
iter: 15 i:17, pairs changed 1
j not moving enough
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
iteration number: 3
j not moving enough
j not moving enough
j not moving enough
iteration number: 4
j not moving enough
j not moving enough
j not moving enough
iteration number: 5
j not moving enough
j not moving enough
j not moving enough
iteration number: 6
j not moving enough
j not moving enough
j not moving enough
iteration number: 7
j not moving enough
j not moving enough
j not moving enough
iteration number: 8
j not moving enough
j not moving enough
j not moving enough
iteration number: 9
j not moving enough
j not moving enough
j not moving enough
iteration number: 10
j not moving enough
j not moving enough
j not moving enough
iterati

j not moving enough
iteration number: 4
j not moving enough
L==H
j not moving enough
j not moving enough
iteration number: 5
j not moving enough
L==H
iter: 5 i:29, pairs changed 1
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
iteration number: 3
j not moving enough
j not moving enough
j not moving enough
iteration number: 4
j not moving enough
j not moving enough
j not moving enough
iteration number: 5
j not moving enough
iter: 5 i:54, pairs changed 1
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
iteration number: 3
j not moving enough
j not moving enough
iteration number: 4
j not moving enough
j not movi

iteration number: 4
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 5
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 6
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 7
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 8
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 9
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 10
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 11
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 12
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 13
iter: 13 i:17, pairs changed 1
j not moving enough
j not moving enough
j not

j not moving enough
iter: 3 i:55, pairs changed 1
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
iter: 1 i:54, pairs changed 1
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 3
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 4
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 5
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 6
j not moving enough
iter: 6 i:23, pairs changed 1
j not moving enough
j not moving enough
j not movi

j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 20
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 21
j not moving enough
j not moving enough
L==H
j not moving enough
j not mo

j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 7
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 8
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 9
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough


j not moving enough
j not moving enough
j not moving enough
iteration number: 11
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 12
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 13
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 14
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 15
j not moving enough
iter: 15 i:29, pairs changed 1
j not moving enough
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
iteration number: 3
j not moving enough
j not moving enough
j not moving enough
iteration number: 4
j not moving enough
j not moving enough
j not moving enough
iter

In [9]:
b

matrix([[-3.86795751]])

In [10]:
alphas[alphas > 0]

matrix([[1.27006226e-01, 7.80625564e-17, 2.43410021e-01, 3.70410247e-01]])

In [11]:
np.shape(alphas[alphas > 0])

(1, 4)

In [12]:
for i in range(100):
    if alphas[i] > 0.0:
        print(dataArr[i], labelArr[i])

[4.658191, 3.507396] -1.0
[7.286357, 0.251077] 1.0
[3.457096, -0.082216] -1.0
[6.080573, 0.418886] 1.0


## Platt SMO

### 支持函数

In [13]:
class optStructK:
    def __init__(self ,dataMatIn, classLabels, C, toler, kTup): 
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = np.shape(dataMatIn)[0]
        self.alphas = np.mat(np.zeros((self.m, 1)))
        self.b = 0
        self.eCache = np.mat(np.zeros((self.m, 2)))

In [14]:
def calcEkK(oS, k):
    fXk = float(np.multiply(oS.alphas, oS.labelMat).T * (oS.X * oS.X[k, :].T)) + oS.b
    Ek = fXk - float(oS.labelMat[k])
    return Ek

In [15]:
def selectJK(i, oS, Ei):     
    maxK = -1
    maxDeltaE = 0
    Ej = 0
    oS.eCache[i] = [1, Ei] 
    validEcacheList = np.nonzero(oS.eCache[:, 0].A)[0]
    if len(validEcacheList) > 1:
        for k in validEcacheList:  
            if k == i: 
                continue 
            Ek = calcEkK(oS, k)
            deltaE = abs(Ei - Ek)
            if deltaE > maxDeltaE:
                maxK = k
                maxDeltaE = deltaE
                Ej = Ek
        return maxK, Ej
    else:   
        j = selectJrand(i, oS.m)
        Ej = calcEkK(oS, j)
    return j, Ej

In [16]:
def updateEkK(oS, k):
    Ek = calcEkK(oS, k)
    oS.eCache[k] = [1, Ek]

### 优化例程

In [17]:
def innerLK(i, oS):
    Ei = calcEkK(oS, i)
    if ((oS.labelMat[i] * Ei < - oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i] * Ei > oS.tol) and (oS.alphas[i] > 0)):
        j, Ej = selectJK(i, oS, Ei) 
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        if not oS.labelMat[i] == oS.labelMat[j]:
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L==H: 
            print("L==H")
            return 0
        eta = 2.0 * oS.X[i, :] * oS.X[j, :].T - oS.X[i, :] * oS.X[i, :].T - oS.X[j, :] * oS.X[j, :].T 
        if eta >= 0: 
            print("eta>=0")
            return 0
        oS.alphas[j] -= oS.labelMat[j] * (Ei - Ej) / eta
        oS.alphas[j] = clipAlpha(oS.alphas[j], H, L)
        updateEkK(oS, j)
        if (abs(oS.alphas[j] - alphaJold) < 0.00001): 
            print("j not moving enough")
            return 0
        oS.alphas[i] += oS.labelMat[j] * oS.labelMat[i] * (alphaJold - oS.alphas[j])
        updateEkK(oS, i) 
        b1 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.X[i, :] * oS.X[i, :].T - oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.X[i, :] * oS.X[j, :].T 
        b2 = oS.b - Ej - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.X[i, :] * oS.X[j, :].T - oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.X[j, :] * oS.X[j, :].T 
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): 
            oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): 
            oS.b = b2
        else: 
            oS.b = (b1 + b2) / 2.0
        return 1
    else: 
        return 0

### 外循环

In [18]:
def smoPK(dataMatIn, classLabels, C, toler, maxIter, kTup=('lin', 0)):  
    oS = optStructK(np.mat(dataMatIn), np.mat(classLabels).T, C, toler, kTup)
    iter = 0
    entireSet = True
    alphaPairsChanged = 0
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0
        if entireSet: 
            for i in range(oS.m):        
                alphaPairsChanged += innerLK(i,oS)
                print("fullSet, iter: %d i:%d, pairs changed %d" % (iter, i, alphaPairsChanged))
            iter += 1
        else:
            nonBoundIs = np.nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerLK(i,oS)
                print("non-bound, iter: %d i:%d, pairs changed %d" % (iter, i, alphaPairsChanged))
            iter += 1
        if entireSet: entireSet = False 
        elif (alphaPairsChanged == 0): 
            entireSet = True  
        print("iteration number: %d" % iter)
    return oS.b, oS.alphas

In [19]:
b, alphas = smoPK(dataArr, labelArr, 0.6, 0.001, 40)

L==H
fullSet, iter: 0 i:0, pairs changed 0
L==H
fullSet, iter: 0 i:1, pairs changed 0
fullSet, iter: 0 i:2, pairs changed 1
L==H
fullSet, iter: 0 i:3, pairs changed 1
fullSet, iter: 0 i:4, pairs changed 2
fullSet, iter: 0 i:5, pairs changed 2
fullSet, iter: 0 i:6, pairs changed 2
j not moving enough
fullSet, iter: 0 i:7, pairs changed 2
L==H
fullSet, iter: 0 i:8, pairs changed 2
fullSet, iter: 0 i:9, pairs changed 2
L==H
fullSet, iter: 0 i:10, pairs changed 2
L==H
fullSet, iter: 0 i:11, pairs changed 2
L==H
fullSet, iter: 0 i:12, pairs changed 2
fullSet, iter: 0 i:13, pairs changed 2
L==H
fullSet, iter: 0 i:14, pairs changed 2
fullSet, iter: 0 i:15, pairs changed 2
fullSet, iter: 0 i:16, pairs changed 2
L==H
fullSet, iter: 0 i:17, pairs changed 2
fullSet, iter: 0 i:18, pairs changed 3
fullSet, iter: 0 i:19, pairs changed 3
fullSet, iter: 0 i:20, pairs changed 3
fullSet, iter: 0 i:21, pairs changed 3
j not moving enough
fullSet, iter: 0 i:22, pairs changed 3
L==H
fullSet, iter: 0 i:23, 

In [20]:
def calcWs(alphas,dataArr,classLabels):
    X = np.mat(dataArr)
    labelMat = np.mat(classLabels).T
    m, n = np.shape(X)
    w = np.zeros((n, 1))
    for i in range(m):
        w += np.multiply(alphas[i] * labelMat[i], X[i, :].T)
    return w

In [21]:
ws = calcWs(alphas, dataArr, labelArr)
ws

array([[ 0.65307162],
       [-0.17196128]])

In [22]:
dataMat = np.mat(dataArr)
dataMat[0] * np.mat(ws) + b

matrix([[-0.92555695]])

In [23]:
labelArr[0]

-1.0

In [24]:
dataMat[2] * np.mat(ws) + b

matrix([[2.30436336]])

In [25]:
labelArr[2]

1.0

In [26]:
dataMat[1] * np.mat(ws) + b

matrix([[-1.36706674]])

In [27]:
labelArr[1]

-1.0

## 核函数

### 核转换函数

In [28]:
def kernelTrans(X, A, kTup):
    m, n = np.shape(X)
    K = np.mat(np.zeros((m, 1)))
    if kTup[0] == 'lin': 
        K = X * A.T 
    elif kTup[0] == 'rbf':
        for j in range(m):
            deltaRow = X[j, :] - A
            K[j] = deltaRow * deltaRow.T
        K = np.exp(K / (-1 * kTup[1] ** 2)) 
    else: 
        raise NameError('Houston We Have a Problem -- \
    That Kernel is not recognized')
    return K

In [29]:
class optStruct:
    def __init__(self,dataMatIn, classLabels, C, toler, kTup):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = np.shape(dataMatIn)[0]
        self.alphas = np.mat(np.zeros((self.m, 1)))
        self.b = 0
        self.eCache = np.mat(np.zeros((self.m, 2))) 
        self.K = np.mat(np.zeros((self.m, self.m)))
        for i in range(self.m):
            self.K[:, i] = kernelTrans(self.X, self.X[i, :], kTup)

In [30]:
def calcEk(oS, k):
    fXk = float(np.multiply(oS.alphas, oS.labelMat).T * oS.K[:, k] + oS.b)
    Ek = fXk - float(oS.labelMat[k])
    return Ek

In [31]:
def innerL(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.labelMat[i] * Ei < - oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i] * Ei > oS.tol) and (oS.alphas[i] > 0)):
        j, Ej = selectJ(i, oS, Ei) 
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        if not oS.labelMat[i] == oS.labelMat[j]:
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L==H: 
            print("L==H")
            return 0
        eta = 2.0 * oS.K[i,j] - oS.K[i,i] - oS.K[j,j]
        if eta >= 0: 
            print("eta>=0")
            return 0
        oS.alphas[j] -= oS.labelMat[j] * (Ei - Ej) / eta
        oS.alphas[j] = clipAlpha(oS.alphas[j], H, L)
        updateEk(oS, j)
        if (abs(oS.alphas[j] - alphaJold) < 0.00001): 
            print("j not moving enough")
            return 0
        oS.alphas[i] += oS.labelMat[j] * oS.labelMat[i] * (alphaJold - oS.alphas[j])
        updateEk(oS, i) 
        b1 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.K[i, i] - oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.K[i, j]
        b2 = oS.b - Ej - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.K[i, j] - oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.K[j, j]
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): 
            oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): 
            oS.b = b2
        else: 
            oS.b = (b1 + b2) / 2.0
        return 1
    else: 
        return 0

In [32]:
def updateEk(oS, k):
    Ek = calcEk(oS, k)
    oS.eCache[k] = [1, Ek]

In [33]:
def selectJ(i, oS, Ei): 
    maxK = -1
    maxDeltaE = 0
    Ej = 0
    oS.eCache[i] = [1, Ei] 
    validEcacheList = np.nonzero(oS.eCache[:, 0].A)[0]
    if len(validEcacheList) > 1:
        for k in validEcacheList: 
            if k == i: 
                continue 
            Ek = calcEk(oS, k)
            deltaE = abs(Ei - Ek)
            if deltaE > maxDeltaE:
                maxK = k
                maxDeltaE = deltaE
                Ej = Ek
        return maxK, Ej
    else: 
        j = selectJrand(i, oS.m)
        Ej = calcEk(oS, j)
    return j, Ej

In [34]:
def smoP(dataMatIn, classLabels, C, toler, maxIter, kTup=('lin', 0)):
    oS = optStruct(np.mat(dataMatIn), np.mat(classLabels).T, C, toler, kTup)
    iter = 0
    entireSet = True
    alphaPairsChanged = 0
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0
        if entireSet: 
            for i in range(oS.m):        
                alphaPairsChanged += innerL(i, oS)
                print("fullSet, iter: %d i:%d, pairs changed %d" % (iter, i, alphaPairsChanged))
            iter += 1
        else:
            nonBoundIs = np.nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i,oS)
                print("non-bound, iter: %d i:%d, pairs changed %d" % (iter, i, alphaPairsChanged))
            iter += 1
        if entireSet: 
            entireSet = False 
        elif alphaPairsChanged == 0: 
            entireSet = True  
        print("iteration number: %d" % iter)
    return oS.b, oS.alphas

### 利用核函数进行分类的径向基测试函数

In [35]:
def testRbf(k1=1.3):
    dataArr, labelArr = loadDataSet('testSetRBF.txt')
    b, alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, ('rbf', k1)) 
    datMat = np.mat(dataArr)
    labelMat = np.mat(labelArr).T
    svInd = np.nonzero(alphas.A > 0)[0]
    sVs = datMat[svInd] 
    labelSV = labelMat[svInd];
    print("there are %d Support Vectors" % np.shape(sVs)[0])
    m, n = np.shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i, :], ('rbf', k1))
        predict = kernelEval.T * np.multiply(labelSV,alphas[svInd]) + b
        if not np.sign(predict) == np.sign(labelArr[i]): 
            errorCount += 1
    print("the training error rate is: %f" % (float(errorCount) / m))
    dataArr, labelArr = loadDataSet('testSetRBF2.txt')
    errorCount = 0
    datMat = np.mat(dataArr)
    labelMat = np.mat(labelArr).T
    m, n = np.shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs, datMat[i, :], ('rbf', k1))
        predict = kernelEval.T * np.multiply(labelSV, alphas[svInd]) + b
        if not np.sign(predict) == np.sign(labelArr[i]): 
            errorCount += 1    
    print("the test error rate is: %f" % (float(errorCount) / m))

In [36]:
testRbf()

fullSet, iter: 0 i:0, pairs changed 1
fullSet, iter: 0 i:1, pairs changed 2
fullSet, iter: 0 i:2, pairs changed 3
fullSet, iter: 0 i:3, pairs changed 4
fullSet, iter: 0 i:4, pairs changed 5
fullSet, iter: 0 i:5, pairs changed 6
fullSet, iter: 0 i:6, pairs changed 7
fullSet, iter: 0 i:7, pairs changed 7
fullSet, iter: 0 i:8, pairs changed 8
fullSet, iter: 0 i:9, pairs changed 8
fullSet, iter: 0 i:10, pairs changed 9
fullSet, iter: 0 i:11, pairs changed 10
L==H
fullSet, iter: 0 i:12, pairs changed 10
L==H
fullSet, iter: 0 i:13, pairs changed 10
L==H
fullSet, iter: 0 i:14, pairs changed 10
fullSet, iter: 0 i:15, pairs changed 11
fullSet, iter: 0 i:16, pairs changed 12
fullSet, iter: 0 i:17, pairs changed 12
fullSet, iter: 0 i:18, pairs changed 13
fullSet, iter: 0 i:19, pairs changed 14
fullSet, iter: 0 i:20, pairs changed 14
fullSet, iter: 0 i:21, pairs changed 15
fullSet, iter: 0 i:22, pairs changed 15
fullSet, iter: 0 i:23, pairs changed 15
fullSet, iter: 0 i:24, pairs changed 16
j not 

j not moving enough
fullSet, iter: 5 i:27, pairs changed 0
j not moving enough
fullSet, iter: 5 i:28, pairs changed 0
j not moving enough
fullSet, iter: 5 i:29, pairs changed 0
L==H
fullSet, iter: 5 i:30, pairs changed 0
j not moving enough
fullSet, iter: 5 i:31, pairs changed 0
fullSet, iter: 5 i:32, pairs changed 0
j not moving enough
fullSet, iter: 5 i:33, pairs changed 0
j not moving enough
fullSet, iter: 5 i:34, pairs changed 0
fullSet, iter: 5 i:35, pairs changed 0
fullSet, iter: 5 i:36, pairs changed 0
fullSet, iter: 5 i:37, pairs changed 0
fullSet, iter: 5 i:38, pairs changed 0
fullSet, iter: 5 i:39, pairs changed 0
j not moving enough
fullSet, iter: 5 i:40, pairs changed 0
j not moving enough
fullSet, iter: 5 i:41, pairs changed 0
j not moving enough
fullSet, iter: 5 i:42, pairs changed 0
fullSet, iter: 5 i:43, pairs changed 0
fullSet, iter: 5 i:44, pairs changed 0
j not moving enough
fullSet, iter: 5 i:45, pairs changed 0
fullSet, iter: 5 i:46, pairs changed 0
fullSet, iter: 

## 手写数字识别回顾

In [37]:
def img2vector(filename):
    returnVect = np.zeros((1, 1024))
    fr = open(filename).readline()
    for i in range(32):
        lineStr = fr
        for j in range(32):
            returnVect[0, 32 * i + j] = int(lineStr[j])
    return returnVect

In [38]:
def loadImages(dirName):
    from os import listdir
    hwLabels = []
    trainingFileList = listdir(dirName) 
    m = len(trainingFileList)
    trainingMat = np.zeros((m, 1024))
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0] 
        classNumStr = int(fileStr.split('_')[0])
        if classNumStr == 9: 
            hwLabels.append(-1)
        else: 
            hwLabels.append(1)
        trainingMat[i, :] = img2vector('%s/%s' % (dirName, fileNameStr))
    return trainingMat, hwLabels  

In [39]:
def testDigits(kTup=('rbf', 10)):
    dataArr, labelArr = loadImages('trainingDigits')
    b,alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, kTup)
    datMat = np.mat(dataArr)
    labelMat = np.mat(labelArr).T
    svInd = np.nonzero(alphas.A > 0)[0]
    sVs = datMat[svInd] 
    labelSV = labelMat[svInd];
    print("there are %d Support Vectors" % np.shape(sVs)[0])
    m, n = np.shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs, datMat[i, :], kTup)
        predict = kernelEval.T * np.multiply(labelSV, alphas[svInd]) + b
        if not np.sign(predict) == np.sign(labelArr[i]): 
            errorCount += 1
    print("the training error rate is: %f" % (float(errorCount) / m))
    dataArr, labelArr = loadImages('testDigits')
    errorCount = 0
    datMat = np.mat(dataArr)
    labelMat = np.mat(labelArr).T
    m, n = np.shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs, datMat[i, :], kTup)
        predict = kernelEval.T * np.multiply(labelSV, alphas[svInd]) + b
        if not np.sign(predict) == np.sign(labelArr[i]): 
            errorCount += 1    
    print("the test error rate is: %f" % (float(errorCount) / m) )

In [40]:
testDigits(('rbf', 10))

fullSet, iter: 0 i:0, pairs changed 1
fullSet, iter: 0 i:1, pairs changed 2
fullSet, iter: 0 i:2, pairs changed 3
fullSet, iter: 0 i:3, pairs changed 3
fullSet, iter: 0 i:4, pairs changed 4
fullSet, iter: 0 i:5, pairs changed 5
fullSet, iter: 0 i:6, pairs changed 6
fullSet, iter: 0 i:7, pairs changed 7
fullSet, iter: 0 i:8, pairs changed 8
fullSet, iter: 0 i:9, pairs changed 9
fullSet, iter: 0 i:10, pairs changed 9
fullSet, iter: 0 i:11, pairs changed 10
fullSet, iter: 0 i:12, pairs changed 11
fullSet, iter: 0 i:13, pairs changed 11
fullSet, iter: 0 i:14, pairs changed 12
fullSet, iter: 0 i:15, pairs changed 13
fullSet, iter: 0 i:16, pairs changed 14
fullSet, iter: 0 i:17, pairs changed 14
fullSet, iter: 0 i:18, pairs changed 15
L==H
fullSet, iter: 0 i:19, pairs changed 15
fullSet, iter: 0 i:20, pairs changed 16
fullSet, iter: 0 i:21, pairs changed 16
fullSet, iter: 0 i:22, pairs changed 17
fullSet, iter: 0 i:23, pairs changed 18
fullSet, iter: 0 i:24, pairs changed 18
fullSet, iter: 0

eta>=0
fullSet, iter: 0 i:213, pairs changed 151
fullSet, iter: 0 i:214, pairs changed 152
fullSet, iter: 0 i:215, pairs changed 153
fullSet, iter: 0 i:216, pairs changed 153
fullSet, iter: 0 i:217, pairs changed 154
fullSet, iter: 0 i:218, pairs changed 155
fullSet, iter: 0 i:219, pairs changed 155
fullSet, iter: 0 i:220, pairs changed 156
fullSet, iter: 0 i:221, pairs changed 157
fullSet, iter: 0 i:222, pairs changed 158
fullSet, iter: 0 i:223, pairs changed 159
fullSet, iter: 0 i:224, pairs changed 160
fullSet, iter: 0 i:225, pairs changed 161
fullSet, iter: 0 i:226, pairs changed 162
fullSet, iter: 0 i:227, pairs changed 163
fullSet, iter: 0 i:228, pairs changed 164
fullSet, iter: 0 i:229, pairs changed 165
fullSet, iter: 0 i:230, pairs changed 166
j not moving enough
fullSet, iter: 0 i:231, pairs changed 166
fullSet, iter: 0 i:232, pairs changed 167
fullSet, iter: 0 i:233, pairs changed 167
fullSet, iter: 0 i:234, pairs changed 168
j not moving enough
fullSet, iter: 0 i:235, pairs

non-bound, iter: 1 i:7, pairs changed 7
non-bound, iter: 1 i:8, pairs changed 8
non-bound, iter: 1 i:9, pairs changed 9
non-bound, iter: 1 i:11, pairs changed 10
non-bound, iter: 1 i:12, pairs changed 11
non-bound, iter: 1 i:14, pairs changed 12
non-bound, iter: 1 i:15, pairs changed 13
non-bound, iter: 1 i:16, pairs changed 14
non-bound, iter: 1 i:18, pairs changed 15
non-bound, iter: 1 i:19, pairs changed 16
non-bound, iter: 1 i:20, pairs changed 17
non-bound, iter: 1 i:22, pairs changed 18
non-bound, iter: 1 i:23, pairs changed 19
non-bound, iter: 1 i:27, pairs changed 20
non-bound, iter: 1 i:28, pairs changed 21
non-bound, iter: 1 i:29, pairs changed 22
non-bound, iter: 1 i:30, pairs changed 23
non-bound, iter: 1 i:31, pairs changed 24
non-bound, iter: 1 i:32, pairs changed 25
non-bound, iter: 1 i:33, pairs changed 26
non-bound, iter: 1 i:34, pairs changed 27
non-bound, iter: 1 i:35, pairs changed 28
non-bound, iter: 1 i:36, pairs changed 29
non-bound, iter: 1 i:37, pairs changed 3

non-bound, iter: 1 i:269, pairs changed 200
non-bound, iter: 1 i:270, pairs changed 201
non-bound, iter: 1 i:271, pairs changed 202
non-bound, iter: 1 i:276, pairs changed 203
non-bound, iter: 1 i:280, pairs changed 204
non-bound, iter: 1 i:281, pairs changed 205
non-bound, iter: 1 i:282, pairs changed 206
non-bound, iter: 1 i:283, pairs changed 207
non-bound, iter: 1 i:284, pairs changed 208
non-bound, iter: 1 i:285, pairs changed 209
non-bound, iter: 1 i:286, pairs changed 210
non-bound, iter: 1 i:287, pairs changed 211
non-bound, iter: 1 i:288, pairs changed 212
non-bound, iter: 1 i:290, pairs changed 213
non-bound, iter: 1 i:293, pairs changed 214
non-bound, iter: 1 i:294, pairs changed 215
non-bound, iter: 1 i:295, pairs changed 216
non-bound, iter: 1 i:297, pairs changed 217
non-bound, iter: 1 i:298, pairs changed 218
non-bound, iter: 1 i:299, pairs changed 219
non-bound, iter: 1 i:300, pairs changed 220
non-bound, iter: 1 i:301, pairs changed 221
non-bound, iter: 1 i:302, pairs 

non-bound, iter: 2 i:143, pairs changed 99
non-bound, iter: 2 i:144, pairs changed 100
non-bound, iter: 2 i:145, pairs changed 101
non-bound, iter: 2 i:148, pairs changed 102
non-bound, iter: 2 i:151, pairs changed 103
non-bound, iter: 2 i:152, pairs changed 104
non-bound, iter: 2 i:153, pairs changed 105
non-bound, iter: 2 i:154, pairs changed 106
non-bound, iter: 2 i:159, pairs changed 107
non-bound, iter: 2 i:160, pairs changed 108
non-bound, iter: 2 i:161, pairs changed 109
non-bound, iter: 2 i:162, pairs changed 110
non-bound, iter: 2 i:163, pairs changed 111
non-bound, iter: 2 i:164, pairs changed 112
non-bound, iter: 2 i:165, pairs changed 113
non-bound, iter: 2 i:166, pairs changed 114
non-bound, iter: 2 i:167, pairs changed 115
non-bound, iter: 2 i:168, pairs changed 116
non-bound, iter: 2 i:169, pairs changed 117
non-bound, iter: 2 i:172, pairs changed 118
non-bound, iter: 2 i:173, pairs changed 119
non-bound, iter: 2 i:174, pairs changed 120
non-bound, iter: 2 i:175, pairs c

...

j not moving enough
non-bound, iter: 10 i:88, pairs changed 0
j not moving enough
non-bound, iter: 10 i:90, pairs changed 0
j not moving enough
non-bound, iter: 10 i:93, pairs changed 0
j not moving enough
non-bound, iter: 10 i:95, pairs changed 0
j not moving enough
non-bound, iter: 10 i:96, pairs changed 0
j not moving enough
non-bound, iter: 10 i:97, pairs changed 0
j not moving enough
non-bound, iter: 10 i:98, pairs changed 0
j not moving enough
non-bound, iter: 10 i:100, pairs changed 0
j not moving enough
non-bound, iter: 10 i:101, pairs changed 0
j not moving enough
non-bound, iter: 10 i:103, pairs changed 0
j not moving enough
non-bound, iter: 10 i:104, pairs changed 0
j not moving enough
non-bound, iter: 10 i:105, pairs changed 0
j not moving enough
non-bound, iter: 10 i:106, pairs changed 0
j not moving enough
non-bound, iter: 10 i:108, pairs changed 0
j not moving enough
non-bound, iter: 10 i:109, pairs changed 0
j not moving enough
non-bound, iter: 10 i:111, pairs changed 0

j not moving enough
non-bound, iter: 10 i:257, pairs changed 0
j not moving enough
non-bound, iter: 10 i:258, pairs changed 0
j not moving enough
non-bound, iter: 10 i:259, pairs changed 0
j not moving enough
non-bound, iter: 10 i:260, pairs changed 0
j not moving enough
non-bound, iter: 10 i:261, pairs changed 0
j not moving enough
non-bound, iter: 10 i:262, pairs changed 0
j not moving enough
non-bound, iter: 10 i:263, pairs changed 0
j not moving enough
non-bound, iter: 10 i:264, pairs changed 0
j not moving enough
non-bound, iter: 10 i:265, pairs changed 0
j not moving enough
non-bound, iter: 10 i:266, pairs changed 0
j not moving enough
non-bound, iter: 10 i:267, pairs changed 0
j not moving enough
non-bound, iter: 10 i:268, pairs changed 0
j not moving enough
non-bound, iter: 10 i:269, pairs changed 0
j not moving enough
non-bound, iter: 10 i:270, pairs changed 0
j not moving enough
non-bound, iter: 10 i:271, pairs changed 0
j not moving enough
non-bound, iter: 10 i:276, pairs ch

j not moving enough
fullSet, iter: 11 i:18, pairs changed 0
j not moving enough
fullSet, iter: 11 i:19, pairs changed 0
j not moving enough
fullSet, iter: 11 i:20, pairs changed 0
fullSet, iter: 11 i:21, pairs changed 0
j not moving enough
fullSet, iter: 11 i:22, pairs changed 0
fullSet, iter: 11 i:23, pairs changed 0
j not moving enough
fullSet, iter: 11 i:24, pairs changed 0
L==H
fullSet, iter: 11 i:25, pairs changed 0
L==H
fullSet, iter: 11 i:26, pairs changed 0
j not moving enough
fullSet, iter: 11 i:27, pairs changed 0
j not moving enough
fullSet, iter: 11 i:28, pairs changed 0
j not moving enough
fullSet, iter: 11 i:29, pairs changed 0
j not moving enough
fullSet, iter: 11 i:30, pairs changed 0
j not moving enough
fullSet, iter: 11 i:31, pairs changed 0
fullSet, iter: 11 i:32, pairs changed 0
j not moving enough
fullSet, iter: 11 i:33, pairs changed 0
j not moving enough
fullSet, iter: 11 i:34, pairs changed 0
j not moving enough
fullSet, iter: 11 i:35, pairs changed 0
j not movi

j not moving enough
fullSet, iter: 11 i:171, pairs changed 0
j not moving enough
fullSet, iter: 11 i:172, pairs changed 0
j not moving enough
fullSet, iter: 11 i:173, pairs changed 0
j not moving enough
fullSet, iter: 11 i:174, pairs changed 0
j not moving enough
fullSet, iter: 11 i:175, pairs changed 0
j not moving enough
fullSet, iter: 11 i:176, pairs changed 0
j not moving enough
fullSet, iter: 11 i:177, pairs changed 0
j not moving enough
fullSet, iter: 11 i:178, pairs changed 0
j not moving enough
fullSet, iter: 11 i:179, pairs changed 0
j not moving enough
fullSet, iter: 11 i:180, pairs changed 0
j not moving enough
fullSet, iter: 11 i:181, pairs changed 0
j not moving enough
fullSet, iter: 11 i:182, pairs changed 0
j not moving enough
fullSet, iter: 11 i:183, pairs changed 0
j not moving enough
fullSet, iter: 11 i:184, pairs changed 0
j not moving enough
fullSet, iter: 11 i:185, pairs changed 0
j not moving enough
fullSet, iter: 11 i:186, pairs changed 0
fullSet, iter: 11 i:187,

j not moving enough
fullSet, iter: 11 i:312, pairs changed 0
fullSet, iter: 11 i:313, pairs changed 0
j not moving enough
fullSet, iter: 11 i:314, pairs changed 0
j not moving enough
fullSet, iter: 11 i:315, pairs changed 0
j not moving enough
fullSet, iter: 11 i:316, pairs changed 0
j not moving enough
fullSet, iter: 11 i:317, pairs changed 0
j not moving enough
fullSet, iter: 11 i:318, pairs changed 0
j not moving enough
fullSet, iter: 11 i:319, pairs changed 0
j not moving enough
fullSet, iter: 11 i:320, pairs changed 0
fullSet, iter: 11 i:321, pairs changed 0
fullSet, iter: 11 i:322, pairs changed 0
j not moving enough
fullSet, iter: 11 i:323, pairs changed 0
j not moving enough
fullSet, iter: 11 i:324, pairs changed 0
j not moving enough
fullSet, iter: 11 i:325, pairs changed 0
j not moving enough
fullSet, iter: 11 i:326, pairs changed 0
fullSet, iter: 11 i:327, pairs changed 0
j not moving enough
fullSet, iter: 11 i:328, pairs changed 0
j not moving enough
fullSet, iter: 11 i:329